In [29]:
using Oceananigans
using Oceananigans.Units: minute, minutes, hours
using Oceanostics
grid = RectilinearGrid(GPU();
                       size=(32, 32, 32),
                       extent=(128, 128, 64))

using Oceananigans.BuoyancyModels: g_Earth

In [30]:
amplitude = 0.8 # m
wavelength = 60  # m
wavenumber = 2π / wavelength # m⁻¹
frequency = sqrt(g_Earth * wavenumber) # s⁻¹

# The vertical scale over which the Stokes drift of a monochromatic surface wave
# decays away from the surface is `1/2wavenumber`, or
const vertical_scale = wavelength / 4π

# Stokes drift velocity at the surface
const Uˢ = amplitude^2 * wavenumber * frequency # m s⁻¹
uˢ(z) = Uˢ * exp(z / vertical_scale)
∂z_uˢ(z, t) = 1 / vertical_scale * Uˢ * exp(z / vertical_scale)
Qᵘ = -3.72e-5 # m² s⁻², surface kinematic momentum flux

u_boundary_conditions = FieldBoundaryConditions(top = FluxBoundaryCondition(Qᵘ))
Qᵇ = 2.307e-8 # m² s⁻³, surface buoyancy flux
N² = 1.936e-5 # s⁻², initial and bottom buoyancy gradient

b_boundary_conditions = FieldBoundaryConditions(top = FluxBoundaryCondition(Qᵇ),
                                                bottom = GradientBoundaryCondition(N²))

Qᵇ = 2.307e-8 # m² s⁻³, surface buoyancy flux
N² = 1.936e-5 # s⁻², initial and bottom buoyancy gradient
                                                
b_boundary_conditions = FieldBoundaryConditions(top = FluxBoundaryCondition(Qᵇ),
                        bottom = GradientBoundaryCondition(N²))

coriolis = FPlane(f=1e-4) # s⁻¹
model = NonhydrostaticModel(; grid, coriolis,
                            advection = WENO(),
                            timestepper = :RungeKutta3,
                            tracers = :b,
                            buoyancy = BuoyancyTracer(),
                            closure = AnisotropicMinimumDissipation(),
                            stokes_drift = UniformStokesDrift(∂z_uˢ=∂z_uˢ),
                            boundary_conditions = (u=u_boundary_conditions, b=b_boundary_conditions))
Ξ(z) = randn() * exp(z / 4)
initial_mixed_layer_depth = 33 # m
stratification(z) = z < - initial_mixed_layer_depth ? N² * z : N² * (-initial_mixed_layer_depth)

bᵢ(x, y, z) = stratification(z) + 1e-1 * Ξ(z) * N² * model.grid.Lz

u★ = sqrt(abs(Qᵘ))
uᵢ(x, y, z) = u★ * 1e-1 * Ξ(z)
wᵢ(x, y, z) = u★ * 1e-1 * Ξ(z)

set!(model, u=uᵢ, w=wᵢ, b=bᵢ)
simulation = Simulation(model, Δt=45.0, stop_time=4hours)
wizard = TimeStepWizard(cfl=1.0, max_change=1.1, max_Δt=1minute)
simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))


Callback of TimeStepWizard(cfl=1.0, max_Δt=60.0, min_Δt=0.0) on IterationInterval(10)

In [31]:
u, v, w = model.velocities
p = sum(model.pressures)
W = Field(Average(w, dims=(2, 3)))
P = Field(Average(p, dims=(2, 3)))
w_prime = w-W
p_prime = p-P

BinaryOperation at (Center, Center, Center)
├── grid: 32×32×32 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on GPU with 3×3×3 halo
└── tree: 
    - at (Center, Center, Center)
    ├── + at (Center, Center, Center)
    │   ├── 32×32×32 Field{Center, Center, Center} on RectilinearGrid on GPU
    │   └── 32×32×32 Field{Center, Center, Center} on RectilinearGrid on GPU
    └── 32×1×1 Field{Center, Nothing, Nothing} reduced over dims = (2, 3) on RectilinearGrid on GPU

In [32]:
function ZPressureRedistribution(model, w_prime, p_prime)
    return ∂z(w_prime*p_prime) # p is the total kinematic pressure (there's no need for ρ₀)
end

ZPressureRedistribution (generic function with 2 methods)

In [33]:
wp = ZPressureRedistribution(model, w_prime, p_prime)

Derivative at (Center, Center, Center)
├── grid: 32×32×32 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on GPU with 3×3×3 halo
└── tree: 
    ∂zᶜᶜᶜ at (Center, Center, Center) via identity
    └── * at (Center, Center, Face)
        ├── - at (Center, Center, Face)
        │   ├── 32×32×33 Field{Center, Center, Face} on RectilinearGrid on GPU
        │   └── 32×1×1 Field{Center, Nothing, Nothing} reduced over dims = (2, 3) on RectilinearGrid on GPU
        └── - at (Center, Center, Center)
            ├── + at (Center, Center, Center)
            │   ├── 32×32×32 Field{Center, Center, Center} on RectilinearGrid on GPU
            │   └── 32×32×32 Field{Center, Center, Center} on RectilinearGrid on GPU
            └── 32×1×1 Field{Center, Nothing, Nothing} reduced over dims = (2, 3) on RectilinearGrid on GPU

In [34]:
using Printf

function progress(simulation)
    u, v, w = simulation.model.velocities

    # Print a progress message
    msg = @sprintf("i: %04d, t: %s, Δt: %s, umax = (%.1e, %.1e, %.1e) ms⁻¹, wall time: %s\n",
                   iteration(simulation),
                   prettytime(time(simulation)),
                   prettytime(simulation.Δt),
                   maximum(abs, u), maximum(abs, v), maximum(abs, w),
                   prettytime(simulation.run_wall_time))

    @info msg

    return nothing
end

simulation.callbacks[:progress] = Callback(progress, IterationInterval(20))

# I did not set up the output writer part

Callback of progress on IterationInterval(20)

In [35]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /home/zhengwei/.julia/packages/Oceananigans/WvJuZ/src/Simulations/run.jl:173
┌ Info: i: 0000, t: 0 seconds, Δt: 49.500 seconds, umax = (1.3e-03, 4.4e-04, 8.1e-04) ms⁻¹, wall time: 0 seconds
└ @ Main In[34]:14
┌ Info:     ... simulation initialization complete (113.811 ms)
└ @ Oceananigans.Simulations /home/zhengwei/.julia/packages/Oceananigans/WvJuZ/src/Simulations/run.jl:212
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /home/zhengwei/.julia/packages/Oceananigans/WvJuZ/src/Simulations/run.jl:114
┌ Info:     ... initial time step complete (18.180 seconds).
└ @ Oceananigans.Simulations /home/zhengwei/.julia/packages/Oceananigans/WvJuZ/src/Simulations/run.jl:123
┌ Info: i: 0020, t: 17.325 minutes, Δt: 59.895 seconds, umax = (2.6e-02, 1.1e-02, 1.9e-02) ms⁻¹, wall time: 18.501 seconds
└ @ Main In[34]:14
┌ Info: i: 0040, t: 37.307 minutes, Δt: 1 minute, umax = (3.8e-02, 1.1e-02, 1.8e-02) ms⁻¹, wall time: 1